In [ ]:
#Add pip libraries required for this project
%pip install -qU langchain-core langchain-google-genai langchain
#%pip install -qU langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ['GEMINI_API_KEY'] = '' #replace with your Gemini API key

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=os.getenv('GEMINI_API_KEY'))

In [48]:
from IPython.display import display, Markdown

response = model.invoke("Summarize in 150 words about langchain")
display(Markdown(response.content))

LangChain is an open-source framework that simplifies the development of applications powered by large language models (LLMs). It acts as a bridge between LLMs and real-world data, enabling them to access and process information from various sources. This allows for more sophisticated and contextualized interactions with LLMs.

LangChain provides building blocks for building LLM-powered applications, including:

* **Chains:** Sequences of operations that combine LLMs with other components.
* **Indexes:** Mechanisms to access and query external data sources.
* **Agents:** Autonomous systems that use LLMs to make decisions and take actions.

By streamlining the integration of LLMs with data and other tools, LangChain empowers developers to build powerful and innovative applications.


In [50]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

# Define a memory to hold conversation history
memory = ConversationBufferMemory(memory_key="history", input_key="input", return_messages=True)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{input}")
])

print(chat_prompt.to_json())


# Create a conversational chain with the model and memory
conversation = ConversationChain(
    llm=model,
    prompt=chat_prompt,
    verbose=False,
    memory=memory,
    input_key="input",
)

# Start a conversation loop that continues until the user types "exit or quit"
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Ending the conversation....")
        #print(memory.buffer)
        break   

    # Generate response from the conversational chain
    response = conversation.predict(input=user_input)

    try:
        # Display response as plain text
        print("User:", user_input)
        print("AI:", response)
    except Exception as e:
        memory.buffer.clear()
        print(f"An error occurred: {str(e)}")
        print("Please try again or rephrase your query.")
        

{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'], 'kwargs': {'input_variables': ['history', 'input'], 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. Answer all questions to the best of your ability.'), additional_kwargs={}), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]}, 'name': 'ChatPromptTemplate'}
User: Hello, Ranjit here
AI: Hello Ranjit! 👋  It's nice to meet you. How can I help you today? 😊 

User: How are you doing today
AI: I'm doing well, thank you for asking! 😊  I'm always ready to help with any questions or tasks you have.  What about you, how is your day going? 

User: You remember my name
AI: That's right! I do remember your name, Ranjit.  I'm still learning and 

In [51]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

#build a receipie prompt
recepie_prompt = PromptTemplate.from_template(
    "Create a recipe using the following ingredients or idea: {ingredient}."  
    "Consider the user's dietary preferences {dietary_preferences}."  
)

recepie_chain = LLMChain(
    llm=model,
    prompt=recepie_prompt,
    output_key="recipe",
    verbose=False
)

#build a shopping list prompt
shopping_list_prompt = PromptTemplate.from_template(
    "Based on the following receipe, Create a shopping list of ingredients: \n\n {recipe}."
)

shopping_list_chain = LLMChain(
    llm=model,
    prompt=shopping_list_prompt,
    #input_keys=["recipe"],
    output_key="shopping_list",
    verbose=False
)

#Combine the recepie and shopping list chains 
receipie_sequential_chain = SequentialChain(
    chains=[recepie_chain, shopping_list_chain],
    input_variables=["ingredient", "dietary_preferences"],
    output_variables=["recipe", "shopping_list"],
    verbose=False
)

# prompt the user for the ingredients and dietary preferences
ingredient = input("Enter the ingredients: ")
dietary_preferences = input("Enter the dietary preferences: ")

# Create dictionary of input values
input_data = {
    "ingredient": ingredient,
    "dietary_preferences": dietary_preferences
}

#run the sequential chain with the input data
output_data = receipie_sequential_chain(input_data)

#display the recipe and shopping list
print(display(Markdown("\n Generated Receipie: \n")), display(Markdown(output_data["recipe"])))
print(display(Markdown("\n Shopping List: \n")), display(Markdown(output_data["shopping_list"])))



 Generated Receipie: 


##  Creamy Butter Chicken with a Kick (Non-Vegetarian)

This recipe offers a classic Butter Chicken experience with a touch of spice and a creamy, rich sauce. It's perfect for those who enjoy a balance of flavor and heat.

**Yields:** 4 Servings
**Prep Time:** 20 minutes
**Cook Time:** 30 minutes 

**Ingredients:**

**For the Chicken:**

* 1 lb boneless, skinless chicken thighs, cut into 1-inch pieces
* 1 tbsp olive oil
* 1 tsp ground cumin
* 1 tsp ground coriander
* 1/2 tsp turmeric powder
* 1/2 tsp chili powder
* 1/4 tsp garam masala
* Salt to taste

**For the Sauce:**

* 1 large onion, finely chopped
* 2 cloves garlic, minced
* 1 inch ginger, grated
* 1 tbsp tomato paste
* 1 (14.5 oz) can diced tomatoes, undrained
* 1 cup heavy cream
* 1/2 cup water
* 1 tbsp butter
* 1 tsp garam masala
* 1/2 tsp chili powder (adjust to taste)
* Salt to taste
* Fresh cilantro, chopped, for garnish

**Instructions:**

1. **Marinate the Chicken:** In a bowl, combine the chicken pieces with olive oil, cumin, coriander, turmeric, chili powder, garam masala, and salt. Mix well and set aside to marinate for at least 30 minutes. 
2. **Cook the Chicken:** Heat a large skillet over medium heat. Add the marinated chicken and cook until browned on all sides, about 5-7 minutes. Remove the chicken from the skillet and set aside.
3. **Make the Sauce:** In the same skillet, add the chopped onion and cook until softened, about 5 minutes. Add the minced garlic and grated ginger and cook for another minute. Stir in the tomato paste and cook for 1 minute more.
4. **Add the Tomatoes and Cream:** Add the diced tomatoes and water to the skillet. Bring to a simmer and cook for 10 minutes, stirring occasionally. Stir in the heavy cream, butter, garam masala, chili powder, and salt. Simmer for 5 minutes until the sauce thickens. 
5. **Combine Chicken and Sauce:** Return the cooked chicken to the skillet and coat with the sauce. Simmer for 5 minutes until the chicken is cooked through and the sauce is flavorful. 
6. **Serve:** Garnish with chopped cilantro and serve hot with basmati rice, naan bread, or roti.

**Tips:**

* For a milder flavor, reduce the amount of chili powder.
* You can use chicken breasts instead of thighs, but they may dry out more quickly.
* Add a teaspoon of sugar to the sauce for a slightly sweeter flavor.
* For a richer sauce, use full-fat coconut milk instead of heavy cream.
* Feel free to experiment with different spices and herbs to customize your Butter Chicken.

Enjoy this delicious and satisfying Butter Chicken!

None None



 Shopping List: 


##  Butter Chicken Shopping List 

**Produce:**

* 1 large onion
* 2 cloves garlic
* 1 inch ginger
* Fresh cilantro (for garnish)

**Meat:**

* 1 lb boneless, skinless chicken thighs

**Canned Goods:**

* 1 (14.5 oz) can diced tomatoes (undrained)

**Dairy:**

* 1 cup heavy cream
* 1 tbsp butter

**Spices:**

* 1 tbsp ground cumin
* 1 tbsp ground coriander
* 1/2 tsp turmeric powder
* 1/2 tsp chili powder 
* 1/4 tsp garam masala 
* 1 tsp garam masala (for sauce)
* 1/2 tsp chili powder (for sauce) - adjust to taste

**Other:**

* 1 tbsp olive oil
* 1 tbsp tomato paste
* Salt (to taste)

**Optional:**

* Sugar (for a sweeter flavor)
* Full-fat coconut milk (for a richer sauce)
* Basmati rice, naan bread, or roti (for serving) 


None None
